-- Notepad to myself --

# Spark SQL for DataFrames

### Querying DataFrames with Spark SQL

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Spark_SQL") \
    .getOrCreate()

In [2]:
data_path = 'data/'

In [4]:
df2_json_path = data_path + "utilization.json"
df2 = spark.read.json(df2_json_path)

In [ ]:
#df2_csv_path = data_path + "utilization.csv"
#df2 = spark.read.csv(df2_csv_path, header=True, inferSchema=True)

In [7]:
df2.printSchema()

root
 |-- cpu_utilization: double (nullable = true)
 |-- event_datetime: string (nullable = true)
 |-- free_memory: double (nullable = true)
 |-- server_id: long (nullable = true)
 |-- session_count: long (nullable = true)



In [6]:
df2.show(5, truncate=False)

+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|event_datetime     |free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|0.57           |03/05/2019 08:06:14|0.51       |100      |47           |
|0.47           |03/05/2019 08:11:14|0.62       |100      |43           |
|0.56           |03/05/2019 08:16:14|0.57       |100      |62           |
|0.57           |03/05/2019 08:21:14|0.56       |100      |50           |
|0.35           |03/05/2019 08:26:14|0.46       |100      |43           |
+---------------+-------------------+-----------+---------+-------------+
only showing top 5 rows



In [8]:
df2.describe().show()

+-------+-------------------+-------------------+------------------+------------------+------------------+
|summary|    cpu_utilization|     event_datetime|       free_memory|         server_id|     session_count|
+-------+-------------------+-------------------+------------------+------------------+------------------+
|  count|             500000|             500000|            500000|            500000|            500000|
|   mean| 0.6205177400000123|               null|0.3791280999999977|             124.5|          69.59616|
| stddev|0.15875173872912837|               null|0.1583093127837622|14.430884120553253|14.850676696352865|
|    min|               0.22|03/05/2019 08:06:14|               0.0|               100|                32|
|    max|                1.0|04/09/2019 01:22:46|              0.78|               149|               105|
+-------+-------------------+-------------------+------------------+------------------+------------------+



In [9]:
df2.createOrReplaceTempView("utilization")

In [12]:
df_sql = spark.sql("SELECT * \
                   FROM utilization \
                   LIMIT 10")
df_sql.show()

+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|           0.57|03/05/2019 08:06:14|       0.51|      100|           47|
|           0.47|03/05/2019 08:11:14|       0.62|      100|           43|
|           0.56|03/05/2019 08:16:14|       0.57|      100|           62|
|           0.57|03/05/2019 08:21:14|       0.56|      100|           50|
|           0.35|03/05/2019 08:26:14|       0.46|      100|           43|
|           0.41|03/05/2019 08:31:14|       0.58|      100|           48|
|           0.57|03/05/2019 08:36:14|       0.35|      100|           58|
|           0.41|03/05/2019 08:41:14|        0.4|      100|           58|
|           0.53|03/05/2019 08:46:14|       0.35|      100|           62|
|           0.51|03/05/2019 08:51:14|        0.6|      100|           45|
+---------------+-------------------+-

In [11]:
df_sql.count()

10

In [14]:
df_sql = spark.sql("SELECT server_id as sid, session_count as sct \
                   FROM utilization \
                   LIMIT 10")
df_sql.show()

+---+---+
|sid|sct|
+---+---+
|100| 47|
|100| 43|
|100| 62|
|100| 50|
|100| 43|
|100| 48|
|100| 58|
|100| 58|
|100| 62|
|100| 45|
+---+---+



### Filtering DataFrames with Spark SQL

In [16]:
df_sql = spark.sql("SELECT * \
                   FROM utilization \
                   WHERE server_id = 120")
df_sql.show(10, truncate=False)

+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|event_datetime     |free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|0.66           |03/05/2019 08:06:48|0.31       |120      |54           |
|0.58           |03/05/2019 08:11:48|0.38       |120      |64           |
|0.55           |03/05/2019 08:16:48|0.61       |120      |54           |
|0.7            |03/05/2019 08:21:48|0.35       |120      |80           |
|0.6            |03/05/2019 08:26:48|0.39       |120      |71           |
|0.53           |03/05/2019 08:31:48|0.35       |120      |49           |
|0.73           |03/05/2019 08:36:48|0.42       |120      |73           |
|0.41           |03/05/2019 08:41:48|0.6        |120      |72           |
|0.62           |03/05/2019 08:46:48|0.57       |120      |57           |
|0.67           |03/05/2019 08:51:48|0.44       |120      |78           |
+---------------+-------------------+-

In [17]:
df_sql.count()

10000

In [20]:
df_sql = spark.sql("SELECT server_id, session_count \
                   FROM utilization \
                   WHERE session_count > 70")
df_sql.show(5)

+---------+-------------+
|server_id|session_count|
+---------+-------------+
|      100|           71|
|      100|           71|
|      100|           71|
|      100|           71|
|      100|           72|
+---------+-------------+
only showing top 5 rows



In [19]:
df_sql.count()

239659

In [25]:
df_sql = spark.sql("SELECT server_id, session_count \
                   FROM utilization \
                   WHERE session_count > 70 AND server_id = 120")
df_sql.show(5)

+---------+-------------+
|server_id|session_count|
+---------+-------------+
|      120|           80|
|      120|           71|
|      120|           73|
|      120|           72|
|      120|           78|
+---------+-------------+
only showing top 5 rows



In [26]:
df_sql.count()

2733

In [28]:
df_sql = spark.sql("SELECT server_id, session_count \
                    FROM utilization \
                    WHERE session_count > 70 AND server_id = 120 \
                    ORDER BY session_count DESC")
df_sql.show(5)

+---------+-------------+
|server_id|session_count|
+---------+-------------+
|      120|           80|
|      120|           80|
|      120|           80|
|      120|           80|
|      120|           80|
+---------+-------------+
only showing top 5 rows



### Aggregating DataFrames with Spark SQL

In [29]:
df_sql = spark.sql("SELECT count(*) \
                   FROM utilization")
df_sql.show()

+--------+
|count(1)|
+--------+
|  500000|
+--------+



In [30]:
df_sql = spark.sql("SELECT count(*) \
                    FROM utilization \
                    WHERE session_count > 70")
df_sql.show()

+--------+
|count(1)|
+--------+
|  239659|
+--------+



In [32]:
df_sql = spark.sql("SELECT server_id, count(*) \
                    FROM utilization \
                    WHERE session_count > 70 \
                    GROUP BY server_id")
df_sql.show(10)

+---------+--------+
|server_id|count(1)|
+---------+--------+
|      107|    5646|
|      103|    8744|
|      104|    7366|
|      100|     391|
|      105|    1110|
|      108|    8375|
|      101|    9808|
|      102|    8586|
|      112|    7425|
|      113|    9418|
+---------+--------+
only showing top 10 rows



In [33]:
df_sql = spark.sql("SELECT server_id, count(*) \
                    FROM utilization \
                    WHERE session_count > 70 \
                    GROUP BY server_id \
                    ORDER BY count(*) DESC")
df_sql.show(10)

+---------+--------+
|server_id|count(1)|
+---------+--------+
|      101|    9808|
|      113|    9418|
|      145|    9304|
|      103|    8744|
|      102|    8586|
|      133|    8583|
|      108|    8375|
|      149|    8288|
|      137|    8248|
|      148|    8027|
+---------+--------+
only showing top 10 rows



In [35]:
df_sql = spark.sql("SELECT server_id, min(session_count), avg(session_count), max(session_count) \
                    FROM utilization \
                    WHERE session_count > 70 \
                    GROUP BY server_id \
                    ORDER BY count(*) DESC")
df_sql.show(10)

+---------+------------------+------------------+------------------+
|server_id|min(session_count)|avg(session_count)|max(session_count)|
+---------+------------------+------------------+------------------+
|      101|                71| 87.66557911908646|               105|
|      113|                71| 86.96262476109577|               103|
|      145|                71| 86.97732158211522|               103|
|      103|                71| 85.76372369624886|               101|
|      102|                71| 85.71150710458886|               101|
|      133|                71| 85.46720260981009|               100|
|      108|                71|  85.1219104477612|               100|
|      149|                71|  84.9612693050193|                99|
|      137|                71|  85.0061833171678|                99|
|      148|                71| 84.70350068518749|                99|
+---------+------------------+------------------+------------------+
only showing top 10 rows



In [39]:
df_sql = spark.sql("SELECT server_id, min(session_count) min_sc, round(avg(session_count),2) avg_sc, max(session_count) max_sc \
                    FROM utilization \
                    WHERE session_count > 70 \
                    GROUP BY server_id \
                    ORDER BY count(*) DESC")
df_sql.show(10)

+---------+------+------+------+
|server_id|min_sc|avg_sc|max_sc|
+---------+------+------+------+
|      101|    71| 87.67|   105|
|      113|    71| 86.96|   103|
|      145|    71| 86.98|   103|
|      103|    71| 85.76|   101|
|      102|    71| 85.71|   101|
|      133|    71| 85.47|   100|
|      108|    71| 85.12|   100|
|      149|    71| 84.96|    99|
|      137|    71| 85.01|    99|
|      148|    71|  84.7|    99|
+---------+------+------+------+
only showing top 10 rows



### Joining DataFrames with Spark SQL

In [41]:
df2.show(5, truncate=False) #utilization table

+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|event_datetime     |free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|0.57           |03/05/2019 08:06:14|0.51       |100      |47           |
|0.47           |03/05/2019 08:11:14|0.62       |100      |43           |
|0.56           |03/05/2019 08:16:14|0.57       |100      |62           |
|0.57           |03/05/2019 08:21:14|0.56       |100      |50           |
|0.35           |03/05/2019 08:26:14|0.46       |100      |43           |
+---------------+-------------------+-----------+---------+-------------+
only showing top 5 rows



In [42]:
df3_path = data_path + "server_name.csv"
df3 = spark.read.csv(df3_path, header=True, inferSchema=True)

In [44]:
df3.show(5)

+---------+-----------+
|server_id|server_name|
+---------+-----------+
|      100| 100 Server|
|      101| 101 Server|
|      102| 102 Server|
|      103| 103 Server|
|      104| 104 Server|
+---------+-----------+
only showing top 5 rows



In [45]:
df3.createOrReplaceTempView("server_name")

In [48]:
df_sql = spark.sql("SELECT COUNT(DISTINCT server_id) \
                   FROM utilization")
df_sql.show()

+-------------------------+
|count(DISTINCT server_id)|
+-------------------------+
|                       50|
+-------------------------+



In [50]:
df_sql = spark.sql("SELECT min(server_id), max(server_id) \
                   FROM utilization")
df_sql.show()

+--------------+--------------+
|min(server_id)|max(server_id)|
+--------------+--------------+
|           100|           149|
+--------------+--------------+



In [52]:
df_sql = spark.sql("SELECT COUNT(DISTINCT server_id) \
                   FROM server_name")
df_sql.show()

+-------------------------+
|count(DISTINCT server_id)|
+-------------------------+
|                       50|
+-------------------------+



In [53]:
df_sql = spark.sql("SELECT min(server_id), max(server_id) \
                   FROM server_name")
df_sql.show()

+--------------+--------------+
|min(server_id)|max(server_id)|
+--------------+--------------+
|           100|           149|
+--------------+--------------+



In [58]:
df_join = spark.sql("SELECT u.server_id, sn.server_name, u.session_count \
                     FROM utilization u \
                     INNER JOIN server_name sn \
                     ON sn.server_id = u.server_id")
df_join.show(10)

+---------+-----------+-------------+
|server_id|server_name|session_count|
+---------+-----------+-------------+
|      100| 100 Server|           47|
|      100| 100 Server|           43|
|      100| 100 Server|           62|
|      100| 100 Server|           50|
|      100| 100 Server|           43|
|      100| 100 Server|           48|
|      100| 100 Server|           58|
|      100| 100 Server|           58|
|      100| 100 Server|           62|
|      100| 100 Server|           45|
+---------+-----------+-------------+
only showing top 10 rows



In [67]:
df_join = spark.sql("SELECT * \
                     FROM utilization u \
                     LEFT JOIN server_name sn \
                     ON sn.server_id = u.server_id")
df_join.show(10, truncate=False)

+---------------+-------------------+-----------+---------+-------------+---------+-----------+
|cpu_utilization|event_datetime     |free_memory|server_id|session_count|server_id|server_name|
+---------------+-------------------+-----------+---------+-------------+---------+-----------+
|0.57           |03/05/2019 08:06:14|0.51       |100      |47           |100      |100 Server |
|0.47           |03/05/2019 08:11:14|0.62       |100      |43           |100      |100 Server |
|0.56           |03/05/2019 08:16:14|0.57       |100      |62           |100      |100 Server |
|0.57           |03/05/2019 08:21:14|0.56       |100      |50           |100      |100 Server |
|0.35           |03/05/2019 08:26:14|0.46       |100      |43           |100      |100 Server |
|0.41           |03/05/2019 08:31:14|0.58       |100      |48           |100      |100 Server |
|0.57           |03/05/2019 08:36:14|0.35       |100      |58           |100      |100 Server |
|0.41           |03/05/2019 08:41:14|0.4

#### Dropping duplicates

In [70]:
from pyspark.context import SparkContext
sc = SparkContext.getOrCreate()

In [71]:
from pyspark.sql import Row
df_dup = sc.parallelize([Row(server_name='101 Server', cpu_utilization=85, session_count=80), \
                         Row(server_name='101 Server', cpu_utilization=80, session_count=90),
                         Row(server_name='102 Server', cpu_utilization=85, session_count=80),
                         Row(server_name='102 Server', cpu_utilization=85, session_count=80)]).toDF()
df_dup.show()

+-----------+---------------+-------------+
|server_name|cpu_utilization|session_count|
+-----------+---------------+-------------+
| 101 Server|             85|           80|
| 101 Server|             80|           90|
| 102 Server|             85|           80|
| 102 Server|             85|           80|
+-----------+---------------+-------------+



In [72]:
df_dup.drop_duplicates().show()

+-----------+---------------+-------------+
|server_name|cpu_utilization|session_count|
+-----------+---------------+-------------+
| 101 Server|             85|           80|
| 101 Server|             80|           90|
| 102 Server|             85|           80|
+-----------+---------------+-------------+



In [73]:
df_dup.drop_duplicates(['server_name']).show()

+-----------+---------------+-------------+
|server_name|cpu_utilization|session_count|
+-----------+---------------+-------------+
| 101 Server|             85|           80|
| 102 Server|             85|           80|
+-----------+---------------+-------------+



### Working with NAs

In [74]:
from pyspark.sql import Row
from pyspark.sql.functions import lit
from pyspark.sql.types import StringType

In [77]:
df = sc.parallelize([Row(server_name='101 Server', cpu_utilization=85, session_count=80), \
                     Row(server_name='101 Server', cpu_utilization=80, session_count=90),
                     Row(server_name='102 Server', cpu_utilization=85, session_count=40),
                     Row(server_name='103 Server', cpu_utilization=70, session_count=80),
                     Row(server_name='104 Server', cpu_utilization=60, session_count=80)]).toDF()

In [78]:
df.show()

+-----------+---------------+-------------+
|server_name|cpu_utilization|session_count|
+-----------+---------------+-------------+
| 101 Server|             85|           80|
| 101 Server|             80|           90|
| 102 Server|             85|           40|
| 103 Server|             70|           80|
| 104 Server|             60|           80|
+-----------+---------------+-------------+



In [79]:
df_na = df.withColumn('na_col', lit(None).cast(StringType()))
df_na.show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 Server|             85|           80|  null|
| 101 Server|             80|           90|  null|
| 102 Server|             85|           40|  null|
| 103 Server|             70|           80|  null|
| 104 Server|             60|           80|  null|
+-----------+---------------+-------------+------+



In [82]:
df_na.fillna('Z').show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 Server|             85|           80|     Z|
| 101 Server|             80|           90|     Z|
| 102 Server|             85|           40|     Z|
| 103 Server|             70|           80|     Z|
| 104 Server|             60|           80|     Z|
+-----------+---------------+-------------+------+



In [83]:
df_na2 = df_na.fillna('Z').union(df_na)
df_na2.show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 Server|             85|           80|     Z|
| 101 Server|             80|           90|     Z|
| 102 Server|             85|           40|     Z|
| 103 Server|             70|           80|     Z|
| 104 Server|             60|           80|     Z|
| 101 Server|             85|           80|  null|
| 101 Server|             80|           90|  null|
| 102 Server|             85|           40|  null|
| 103 Server|             70|           80|  null|
| 104 Server|             60|           80|  null|
+-----------+---------------+-------------+------+



In [86]:
df_na2.na.drop().show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 Server|             85|           80|     Z|
| 101 Server|             80|           90|     Z|
| 102 Server|             85|           40|     Z|
| 103 Server|             70|           80|     Z|
| 104 Server|             60|           80|     Z|
+-----------+---------------+-------------+------+



In [87]:
df_na2.createOrReplaceTempView("table_wNAs")

In [88]:
spark.sql("SELECT * \
          FROM table_wNAs") \
.show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 Server|             85|           80|     Z|
| 101 Server|             80|           90|     Z|
| 102 Server|             85|           40|     Z|
| 103 Server|             70|           80|     Z|
| 104 Server|             60|           80|     Z|
| 101 Server|             85|           80|  null|
| 101 Server|             80|           90|  null|
| 102 Server|             85|           40|  null|
| 103 Server|             70|           80|  null|
| 104 Server|             60|           80|  null|
+-----------+---------------+-------------+------+



In [90]:
spark.sql("SELECT * \
          FROM table_wNAs \
          WHERE na_col IS NULL") \
.show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 Server|             85|           80|  null|
| 101 Server|             80|           90|  null|
| 102 Server|             85|           40|  null|
| 103 Server|             70|           80|  null|
| 104 Server|             60|           80|  null|
+-----------+---------------+-------------+------+



In [91]:
spark.sql("SELECT * \
          FROM table_wNAs \
          WHERE na_col IS NOT NULL") \
.show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 Server|             85|           80|     Z|
| 101 Server|             80|           90|     Z|
| 102 Server|             85|           40|     Z|
| 103 Server|             70|           80|     Z|
| 104 Server|             60|           80|     Z|
+-----------+---------------+-------------+------+

